# Homework 09: SQL

Total questions: 6<br/>
Total points: 11

### FYI

In class, we used paths on your local machine outside of the notebook to persist data, but here, we'll use a special argument – `:memory:` – that `sqlite3` has which stores the database in-memory -- in otherwords, the database will live only so long as you're running the notebook. Afterwards, it will be thrown away. Usually this argument is useful when experimenting, given that as we discussed, it is persistence that often makes databases useful. We use it here however simply to avoid needing to upload multiple files to Courseworks -- everything you need is here.

If for any reason you need to discard the contents of your database to start over, you may therefore do so by simply restarting the "kernel" of this notebook, which you can do by clicking the restart button in the toolbar (it's 2 buttons to the right of the "Run" button) or in the Kernel menu.

## Question 1

We'll start by creating some tables to store our data -- we'll first have one containing some product information.

Products have:

* a name (`TEXT`)
* a description (`TEXT`)
* a unit cost stored in cents (`INTEGER`)

and of course we also add an id column to identify them.

The SQL to create each table is below. Execute it on the connection provided below to create the table.

```sql
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    unit_cost_cents INTEGER
)
```

[0.5 points]

In [3]:
import sqlite3
connection = sqlite3.connect(":memory:")  # this is our in-memory database, not stored on your hard drive

# Solution
with connection:
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY,
            name TEXT,
            description TEXT,
            unit_cost_cents INTEGER
        )
        """
    )


In [4]:
# autograder tests
with connection:
    result = connection.execute(
        """
        SELECT name FROM sqlite_master WHERE type='table';
        """
    )
    tables = {table for table, in result}
assert "products" in tables

## Question 2

Separately, we'd like to track the number of items in stock for each product.

To do so we'll have a `store` and an `inventory` table.

Stores have just an `id` and a `name`. **Create this table** yourself with a similar `CREATE TABLE` statement to the above.

Then, our `inventory` table should combine stores and products, listing how much of each product each store has in stock.

* a `product_id` (`INTEGER`)
* a `store_id` (`INTEGER`)
* a `quantity` (`INTEGER`) in stock

_(Note: in this case below, we actually don't need a separate `id` column because we can use the combination of `store_id` and `product_id` to identify each row. This is known as a_ composite primary key _and is another fundamental relational database concept, but can be ignored if need be for our purposes)._

Here is a statement creating this table:

```sql
CREATE TABLE IF NOT EXISTS inventory (
    product_id INTEGER NOT NULL,
    store_id INTEGER NOT NULL,
    quantity INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(id),
    FOREIGN KEY (store_id) REFERENCES stores(id),
    PRIMARY KEY (product_id, store_id)
)
```
which you should execute as well.

[0.5 points]

In [5]:
# solution
with connection:
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS stores (
            id INTEGER PRIMARY KEY,
            name TEXT
        )
        """
    )
    connection.execute(
        """
        CREATE TABLE IF NOT EXISTS inventory (
            product_id INTEGER NOT NULL,
            store_id INTEGER NOT NULL,
            quantity INTEGER,
            FOREIGN KEY (product_id) REFERENCES products(id),
            FOREIGN KEY (store_id) REFERENCES stores(id),
            PRIMARY KEY (product_id, store_id)
        )
        """
    )

In [6]:
# autograder tests
with connection:
    result = connection.execute(
        """
        SELECT name FROM sqlite_master WHERE type='table';
        """
    )
    tables = {table for table, in result}
assert all(table in tables for table in {"products", "inventory"})
assert "store" in tables or "stores" in tables

## Question 3

Now we can insert some stores, products and inventory into our database.

There are 2 stores -- one called `NY` and one called `NJ`. There are 2 products we are concerned with. Their names are `sneakers`, costing `$220` (remember this is dollars!) and boots costing `$350`. Use any description for each that you'd like. `NY` has 4 sneakers in stock and 3 boots. `NJ` has 5 sneakers in stock and no boots.

Insert the above data into the tables you have created.

[4 points]

In [7]:
# solution
with connection:
    ny_id = connection.execute('INSERT INTO stores (name) VALUES ("NY")').lastrowid    
    nj_id = connection.execute('INSERT INTO stores (name) VALUES ("NJ")').lastrowid

    connection.executemany(
        """
        INSERT INTO products (name, description, unit_cost_cents)
             VALUES (?, ?, ?)
        """, [
            ("sneakers", "A pair of sneakers", 22000),
            ("boots", "A pair of boots", 35000),
        ],
    )
    
    product_ids = dict(connection.execute("SELECT name, id FROM products"))
    
    connection.executemany(
        """
        INSERT INTO inventory (store_id, product_id, quantity)
             VALUES (?, ?, ?)
        """, [
            (ny_id, product_ids["sneakers"], 4),
            (ny_id, product_ids["boots"], 3),
            (nj_id, product_ids["sneakers"], 5),
            (nj_id, product_ids["boots"], 0),
        ],
    )

In [8]:
# autograder tests
with connection:
    try:
        result = connection.execute(
            """
            SELECT name FROM store
            """
        )
    except sqlite3.OperationalError:
        result = connection.execute(
            """
            SELECT name FROM stores
            """
        )
    stores = {store for store, in result}
assert stores == {"NY", "NJ"}, stores

In [9]:
# autograder tests
with connection:
    result = connection.execute(
        """
        SELECT name, unit_cost_cents FROM products
        """
    ).fetchall()
assert set(result) == {("sneakers", 22000), ("boots", 35000)}, result

In [10]:
# autograder tests
with connection:
    try:
        result = connection.execute(
            """
            SELECT store.name, products.name, quantity
              FROM inventory
              JOIN products ON inventory.product_id = products.id
              JOIN store ON inventory.store_id = store.id
            """
        ).fetchall()
    except sqlite3.OperationalError:
        result = connection.execute(
            """
            SELECT stores.name, products.name, quantity
              FROM inventory
              JOIN products ON inventory.product_id = products.id
              JOIN stores ON inventory.store_id = stores.id
            """
        ).fetchall()
assert set(result) == {
    ("NY", "sneakers", 4), 
    ("NY", "boots", 3), 
    ("NJ", "sneakers", 5), 
    ("NJ", "boots", 0)
}, result

## Question 4

We can of course calculate the total amount of products across all stores.

Write, but do not execute! a `SELECT` query which calculates this sum and returns it as its single returned column. You'll want to use a `SQL` function called `SUM` which functions much like the `COUNT` aggregation function we used in class.

Assign your query to a variable called `query`.

[1.5 point]

In [11]:
# Solution
query = """
SELECT SUM(quantity) FROM inventory
"""

In [12]:
# autograder tests
with connection:
    total, = connection.execute(query).fetchone()
assert total == 12, total

In [13]:
# autograder tests
assert "select" in query.lower()
assert "quantity" in query.lower()

## Question 5

What about the total value _in cents_ of all products in stock? I.e. the `sum of (quantity in stock * cost of product across all products)`?

Create a function called `total_current_value` which returns this `int`. You may calculate this entirely in SQL or partially in Python after retrieving the data you need from the database. You must however retrieve some of the data from the database.

[1.5 points]

In [14]:
# solution
def total_current_value():
    with connection:
        result = connection.execute(
            """
            SELECT store_id, products.unit_cost_cents, quantity
              FROM inventory
              JOIN products ON inventory.product_id = products.id
            """
        )
        return sum(cost * quantity for _, cost, quantity in result)

In [15]:
# autograder tests
assert total_current_value() == 303000, total_current_value()

In [16]:
# autograder tests
assert "connection" in total_current_value.__code__.co_names

## Question 6

Let's say we have a spreadsheet of historical hourly weather information:

| timestamp           | station_code | temp  | conditions |
|---------------------|--------------|-------|------------|
| 2023-03-26 14:00:00 | 12           | 56.4  | Clear Sky  |
| 2023-03-26 13:30:00 | 11           | 42.0  | Few Clouds |
| 2023-03-26 13:00:00 | 12           | 55.8  | Clear Sky  |
| 2023-03-26 12:30:00 | 11           | 41.5  | Overcast   |


We want to put all that historical data into our database, but first: we have to create the table.

Write a string that, when executed, will create a table called `hourly_weather` for our weather information. Assign that string to the variable `weather_schema`. 

Be sure to:
* define & declare a primary key named `id`
* use appropriate data types
* write valid SQL
* use the column titles in the example data as column names in your schema

You can assume the data types deduced in the example will _not_ change. 

Refer to the first column in the table in section [3.1.1 Affinity Name Examples](https://www.sqlite.org/datatype3.html) in SQLite's documentation for better names of data types to use other than the basics it supports. Specifically, any of the following: `INTEGER`, `FLOAT`, `REAL`, `BLOB`, `TEXT`, `BOOLEAN`, `DATE`, and `DATETIME`.

[3 points]

In [17]:
# Solution
weather_schema = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY,
    timestamp DATETIME,
    station_code INTEGER,
    temp FLOAT,
    conditions TEXT
)
"""

In [18]:
# Autograder test - 0.5 points
# Ensure valid SQL is written

# make a new in-memory database
connection = sqlite3.connect(":memory:")

with connection:
    connection.execute(weather_schema)

In [19]:
# Autograder test - 0.5 points
# Ensure the table created has the expected name
QUERY = """
SELECT name 
FROM sqlite_master 
WHERE type='table' AND name NOT LIKE 'sqlite_%'
"""

with connection:
    rows = list(connection.execute(QUERY))
    
assert len(rows) == 1
table = rows[0]
assert len(table) == 1
table_name = table[0]

assert "hourly_weather" == table_name, table_name

In [20]:
# Autograder test - 0.5 points
# Ensure primary key is defined
QUERY = f"PRAGMA table_info({table_name})"

with connection:
    rows = list(connection.execute(QUERY))

assert len(rows) == 5, f"Expected 5 columns, got {len(rows)}."

primary_keys = []
for row in rows:
    if row[-1] == 1:
        primary_keys.append(row[1])
        
assert len(primary_keys) == 1, f"Found {len(primary_keys)} primary key(s), expected 1"
assert primary_keys[0] == "id", f"Primary key should be named 'id', found '{primary_keys[0]}'"

In [21]:
# Autograder test - 0.5 points
# Ensure expected column names
QUERY = f"PRAGMA table_info({table_name})"

with connection:
    rows = list(connection.execute(QUERY))

expected_column_names = sorted(["id", "timestamp", "station_code", "temp", "conditions"])
actual_column_names = sorted([row[1] for row in rows])

assert expected_column_names == actual_column_names, f"Expected: {', '.join(expected_column_names)}\nGot: {', '.join(actual_column_names)}"

In [22]:
# Autograder test - 1 point
# Ensure expected column types
QUERY = f"PRAGMA table_info({table_name})"

with connection:
    rows = list(connection.execute(QUERY))

actual_name_to_types = {row[1]: row[2] for row in rows}
actual_types = sorted([row[2] for row in rows])

assert actual_name_to_types["id"] == "INTEGER", "Expected `INTEGER` for 'id'"
assert actual_name_to_types["timestamp"] in ("DATETIME", "TEXT"), "Expected `DATETIME` or `TEXT` for 'timestamp'"
assert actual_name_to_types["station_code"] == "INTEGER", "Expected `INTEGER` for 'station_code'"
assert actual_name_to_types["temp"] in ("FLOAT", "REAL"), "Expected `FLOAT` or `REAL` for 'temp'"
assert actual_name_to_types["conditions"] == "TEXT", "Expected `TEXT` for 'conditions'"

In [23]:
# Autograder test - 0 points
# Zero point test: suggest to use datetime

msg = (
    "While `DATETIME` and other similar types are not explicitly \n"
    "supported in SQLite, `DATETIME` is supported as an alias to \n"
    "`TEXT`. It's better to use `DATETIME` and other alias types \n"
    "so readers of your code (and future you) understand what is \n"
    "expected for this table.\nSee Affinity Name Examples ("
    "https://www.sqlite.org/datatype3.html) for more info."
)
assert actual_name_to_types["timestamp"] == "DATETIME", msg